In [1]:
import sys
sys.path.append('/Users/zhoupeng/Desktop/Athena')

In [2]:
from athena import Strategy, Backtest
import pandas as pd

In [3]:
MULTIPLE_ASSETS_DATA = pd.DataFrame(
    index=['2023-10-18', '2023-10-19', '2023-10-20'],
    data={
        ('AAA','Open'): [100.0, 200.0, 300.0],
        ('AAA','High'): [100.0, 200.0, 300.0],
        ('AAA','Low'): [100.0, 200.0, 300.0],
        ('AAA','Close'): [100.0, 200.0, 300.0],
        ('AAA','Volume'): [2000000, 2500000, 3000000],
        ('BBB','Open'): [100.0, 200.0, 300.0],
        ('BBB','High'): [100.0, 200.0, 300.0],
        ('BBB','Low'): [100.0, 200.0, 300.0],
        ('BBB','Close'): [100.0, 200.0, 300.0],
        ('BBB','Volume'): [1500000, 1700000, 1600000]
    }
)

In [4]:
MULTIPLE_ASSETS_DATA

AAA                                  BBB                       \
             Open   High    Low  Close   Volume   Open   High    Low  Close   
2023-10-18  100.0  100.0  100.0  100.0  2000000  100.0  100.0  100.0  100.0   
2023-10-19  200.0  200.0  200.0  200.0  2500000  200.0  200.0  200.0  200.0   
2023-10-20  300.0  300.0  300.0  300.0  3000000  300.0  300.0  300.0  300.0   

                     
             Volume  
2023-10-18  1500000  
2023-10-19  1700000  
2023-10-20  1600000

In [5]:
# 检查下能否正确按比例开仓位并且对重复仓位进行合并
# 回测逻辑是先下订单-然后再对订单进行统一更新
# 这是对于多仓的测试
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        if i == 100: # 合并仓位并不影响其他仓位
            print("当前时间", self.date)
            print("可用资金", self.broker.cash)
            self.open(symbol='BBB', size = 1/10, price=record[('BBB','Open')], short=False, is_fractional=True)
            print("买完后可用资金", self.broker.cash)
        
        print("当前时间", self.date)
        print("可用资金", self.broker.cash)
        self.open(symbol='AAA', size = 1/10, price=record[('AAA','Open')], short=False, is_fractional=True)
        print("买完后可用资金", self.broker.cash)

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=1000)
res = backtest.run()

当前时间 2023-10-18
可用资金 1000
买完后可用资金 900.0
当前时间 2023-10-19
可用资金 900.0
买完后可用资金 810.0
当前时间 2023-10-20
可用资金 810.0
买完后可用资金 729.0
所有未平仓持仓已经清算完毕！
回测结束，总资金: 1245.0


In [6]:
# 检查下能否正确按比例开仓位并且对重复仓位进行合并
# 回测逻辑是先下订单-然后再对订单进行统一更新
# 这是对于空仓的测试
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        
        print("当前时间", self.date)
        print("可用资金", self.broker.cash)
        self.open(symbol='AAA', size = 1/10, price=record[('AAA','Open')], short=True, is_fractional=True)
        print("买完后可用资金", self.broker.cash)

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=1000)
res = backtest.run()

当前时间 2023-10-18
可用资金 1000
买完后可用资金 900.0
当前时间 2023-10-19
可用资金 900.0
买完后可用资金 810.0
当前时间 2023-10-20
可用资金 810.0
买完后可用资金 729.0
所有未平仓持仓已经清算完毕！
回测结束，总资金: 755.0


In [7]:
# 检查下order_target_percent功能
# order_target_percent增仓
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        
        print("可供使用用资金", self.broker.cash)
        print("持仓价值", self.broker.assets_value) # 问题是到了新的一天我的持仓价值还没有更新

        if i == 0:
            # 这里出现了个bug，target_value要下1单，但是1被当作分数下了全单
            self.order_target_percent(symbol='AAA', target_percent=0.1, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        
        if i == 2:
            self.order_target_percent(symbol='AAA', target_percent=0.5, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=1000)
res = backtest.run()

可供使用用资金 1000
持仓价值 0.0
买完后可用资金 900.0
可供使用用资金 900.0
持仓价值 100.0
可供使用用资金 900.0
持仓价值 200.0
买完后可用资金 550.0
所有未平仓持仓已经清算完毕！
回测结束，总资金: 1200.0


In [8]:
# 检查下order_target_percent功能
# order_target_percent减仓
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        
        print("可供使用用资金", self.broker.cash)
        print("持仓价值", self.broker.assets_value) # 问题是到了新的一天我的持仓价值还没有更新

        if i == 0:
            # 这里出现了个bug，target_value要下1单，但是1被当作分数下了全单
            self.order_target_percent(symbol='AAA', target_percent=0.5, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        
        if i == 2:
            self.order_target_percent(symbol='AAA', target_percent=0.1, price=record[('AAA','Open')], short=False)
            print("买完后可用资金", self.broker.cash)
        

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=1000)
res = backtest.run()

print("减仓仓位的记录: ", res.trades)

可供使用用资金 1000
持仓价值 0.0
买完后可用资金 500.0
可供使用用资金 500.0
持仓价值 500.0
可供使用用资金 500.0
持仓价值 1000.0
买完后可用资金 1350.0
所有未平仓持仓已经清算完毕！
回测结束，总资金: 2000.0
减仓仓位的记录:  [Trade(symbol='AAA', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=100.0, close_price=300.0, position_size=2.8333333333333335, profit_loss=566.6666666666666, change_pct=2.0, trade_commission=0.0, open_commission=0.0, cumulative_return=1566.6666666666665), Trade(symbol='AAA', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=100.0, close_price=300.0, position_size=2.1666666666666665, profit_loss=433.3333333333333, change_pct=200.0, trade_commission=0.0, open_commission=0.0, cumulative_return=1999.9999999999998)]


In [9]:
MULTIPLE_ASSETS_DATA

AAA                                  BBB                       \
             Open   High    Low  Close   Volume   Open   High    Low  Close   
2023-10-18  100.0  100.0  100.0  100.0  2000000  100.0  100.0  100.0  100.0   
2023-10-19  200.0  200.0  200.0  200.0  2500000  200.0  200.0  200.0  200.0   
2023-10-20  300.0  300.0  300.0  300.0  3000000  300.0  300.0  300.0  300.0   

                     
             Volume  
2023-10-18  1500000  
2023-10-19  1700000  
2023-10-20  1600000

In [10]:
# 以下是一个更完整的测试案例
# 这是一个做多的复杂案例
class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        print(self.date, "开始:")
        print("可供使用用资金: ", self.broker.cash)
        print("持仓价值: ", self.broker.assets_value) # 问题是到了新的一天我的持仓价值还没有更新

        if i == 0:  # 第一天开仓
            self.order_target_percent("AAA", target_percent=0.2, price=record[("AAA", "Close")])
            self.order_target_percent("BBB", target_percent=0.3, price=record[("BBB", "Close")])
            print("买完后可用资金", self.broker.cash)

        elif i == 1:  # 第二天部分加仓或调整仓位
            # 增加 AAA 的仓位
            self.order_target_percent("AAA", target_percent=0.3, price=record[("AAA", "Close")])
            # 调整 BBB 到目标 50%
            self.order_target_percent("BBB", target_percent=0.5, price=record[("BBB", "Close")])
            print("买完后可用资金", self.broker.cash)

        elif i == 2:  # 第三天清仓
            self.order_target_percent("AAA", target_percent=0, price=record[("AAA", "Close")])
            self.order_target_percent("BBB", target_percent=0, price=record[("BBB", "Close")])
            print("卖完后可用资金", self.broker.cash)
            print("卖仓产生的trades: ")
            print(self.broker.trades)

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=10000)
res = backtest.run()


2023-10-18 开始:
可供使用用资金:  10000
持仓价值:  0.0
买完后可用资金 5000.0
2023-10-19 开始:
可供使用用资金:  5000.0
持仓价值:  5000.0
买完后可用资金 2000.0
2023-10-20 开始:
可供使用用资金:  2000.0
持仓价值:  13000.0
卖完后可用资金 21500.0
卖仓产生的trades: 
[Trade(symbol='AAA', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=120.0, close_price=300.0, position_size=25.0, profit_loss=4500.0, change_pct=150.0, trade_commission=0.0, open_commission=0.0, cumulative_return=14500.0), Trade(symbol='BBB', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=125.0, close_price=300.0, position_size=40.0, profit_loss=7000.0, change_pct=140.0, trade_commission=0.0, open_commission=0.0, cumulative_return=21500.0)]
所有未平仓持仓已经清算完毕！
回测结束，总资金: 21500.0


In [11]:
# 同时开多空仓的测试

class SingleAssetStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        print(self.date, "开始:")
        print("可用资金:", self.broker.cash)
        print("持仓价值:", self.broker.assets_value)
        
        # 第一天：同时开多仓和空仓
        if i == 0:
            print("=== 开仓阶段 ===")
            # AAA 多头仓，20%资金开多
            self.order_target_percent("AAA", target_percent=0.2, price=record[("AAA", "Close")], short=False)
            # BBB 空头仓，30%资金开空
            self.order_target_percent("BBB", target_percent=0.3, price=record[("BBB", "Close")], short=True)
            print("买完后资金:", self.broker.cash)

        # 第二天：调仓阶段
        elif i == 1:
            print("=== 调仓阶段 ===")
            # AAA 多头加仓到 40%资金
            self.order_target_percent("AAA", target_percent=0.4, price=record[("AAA", "Close")], short=False)
            # BBB 空头仓，调整到 50%资金
            self.order_target_percent("BBB", target_percent=0.5, price=record[("BBB", "Close")], short=True)
            print("调整后资金:", self.broker.cash)

        # 第三天：清仓阶段
        elif i == 2:
            print("=== 清仓阶段 ===")
            # 清仓 `AAA` 多头仓
            self.order_target_percent("AAA", target_percent=0, price=record[("AAA", "Close")], short=False)
            # 清仓 `BBB` 空头仓
            self.order_target_percent("BBB", target_percent=0, price=record[("BBB", "Close")], short=True)
            print("卖完后资金:", self.broker.cash)
            print("交易记录:")
            for trade in self.broker.trades:
                print(trade)

backtest = Backtest(SingleAssetStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=10000)
res = backtest.run()

2023-10-18 开始:
可用资金: 10000
持仓价值: 0.0
=== 开仓阶段 ===
买完后资金: 5000.0
2023-10-19 开始:
可用资金: 5000.0
持仓价值: 5000.0
=== 调仓阶段 ===
调整后资金: 1000.0
2023-10-20 开始:
可用资金: 1000.0
持仓价值: 8000.0
=== 清仓阶段 ===
卖完后资金: 8000.0
交易记录:
Trade(symbol='AAA', short=False, open_date='2023-10-18', close_date='2023-10-20', open_price=133.33333333333334, close_price=300.0, position_size=30.0, profit_loss=5000.0, change_pct=125.0, trade_commission=0.0, open_commission=0.0, cumulative_return=15000.0)
Trade(symbol='BBB', short=True, open_date='2023-10-18', close_date='2023-10-20', open_price=125.0, close_price=300.0, position_size=40.0, profit_loss=-7000.0, change_pct=-140.0, trade_commission=0.0, open_commission=0.0, cumulative_return=8000.0)
所有未平仓持仓已经清算完毕！
回测结束，总资金: 8000.0


In [12]:
MULTIPLE_ASSETS_DATA

AAA                                  BBB                       \
             Open   High    Low  Close   Volume   Open   High    Low  Close   
2023-10-18  100.0  100.0  100.0  100.0  2000000  100.0  100.0  100.0  100.0   
2023-10-19  200.0  200.0  200.0  200.0  2500000  200.0  200.0  200.0  200.0   
2023-10-20  300.0  300.0  300.0  300.0  3000000  300.0  300.0  300.0  300.0   

                     
             Volume  
2023-10-18  1500000  
2023-10-19  1700000  
2023-10-20  1600000

In [13]:
# 构造的数据集要支持开仓-关仓-调仓
MULTIPLE_ASSETS_DATA = pd.DataFrame(
    index=['2023-10-18', '2023-10-19', '2023-10-20', '2023-10-21'],
    data={
        ('AAA','Close'): [100.0, 200.0, 300.0, 400],
        ('BBB','Close'): [200.0, 300.0, 400.0, 500],
        ('CCC','Close'): [100.0, 300.0, 200.0, 100],
        ('DDD','Close'): [400.0, 300.0, 200.0, 100],
    }
)

MULTIPLE_FACTORS_DATA = pd.DataFrame(
    index=['2023-10-18', '2023-10-19', '2023-10-20', '2023-10-21'],
    data={
        ('AAA','factor'): [4, 4, 4, 2],
        ('BBB','factor'): [1, 1, 2, 1],
        ('CCC','factor'): [2, 2, 1, 2],
        ('DDD','factor'): [2, 2, 2, 4],
    }
)

# 第一天多AAA，空BBB
# 第二天多AAA，空BBB （调仓)
# 第三天多AAA，空CCC （开仓+关仓+调仓）
# 第四天多DDD，空BBB （开仓+关仓+调仓）

In [14]:
from athena import sort_the_factor

class SortedFactorInvestStrategy(Strategy):
    def init(self):
        pass

    def next(self, i, record):
        print("----------------------")
        date = self.data.index[i]
        print(str(date) + ': ')

        current_long_positions, current_short_positions = self.broker.current_position_status()
        print("我目前持仓情况:")
        print("多头仓位:", current_long_positions)
        print("空头仓位:", current_short_positions)

        day_factors = MULTIPLE_FACTORS_DATA.loc[date] # 当天的因子数据
        sorted_factor_series = sort_the_factor(day_factors, 'factor') # 从高到低排因子

        print('因子排名前1的股票:')
        print(sorted_factor_series[:1].index.tolist())
        long_stocks = sorted_factor_series[:1].index.tolist()

        print('因子排名后1的股票:')
        print(sorted_factor_series[-1:].index.tolist())
        short_stocks = sorted_factor_series[-1:].index.tolist()

        target_percent = 0.5

        # 平仓
        for stock in current_long_positions:
            # 对于我目前的所有多头仓位,如果不在新的list里，则平仓
            if stock not in long_stocks:
                print("关多仓:", stock)
                self.close(symbol=stock, price=record[(stock,'Close')])
        for stock in current_short_positions:
            if stock not in short_stocks:
                print("关空仓:", stock)
                self.close(symbol=stock, price=record[(stock,'Close')])
        
        # 这里的cash和assets value都应该被正常更新了

        # 调仓
        for stock in long_stocks:
            print("新开多仓/调仓:", stock)
            self.order_target_percent(stock, target_percent=target_percent, price=record[(stock, "Close")], short=False)
        for stock in short_stocks:
            print("新开空仓/调仓:", stock)
            self.order_target_percent(stock, target_percent=target_percent, price=record[(stock, "Close")], short=True)

        print("----------------------")

backtest = Backtest(SortedFactorInvestStrategy, MULTIPLE_ASSETS_DATA, commission=0, cash=10000)
res = backtest.run()

----------------------
2023-10-18: 
我目前持仓情况:
多头仓位: []
空头仓位: []
因子排名前1的股票:
['AAA']
因子排名后1的股票:
['BBB']
新开多仓/调仓: AAA
新开空仓/调仓: BBB
----------------------
----------------------
2023-10-19: 
我目前持仓情况:
多头仓位: ['AAA']
空头仓位: ['BBB']
因子排名前1的股票:
['AAA']
因子排名后1的股票:
['BBB']
新开多仓/调仓: AAA
新开空仓/调仓: BBB
----------------------
----------------------
2023-10-20: 
我目前持仓情况:
多头仓位: ['AAA']
空头仓位: ['BBB']
因子排名前1的股票:
['AAA']
因子排名后1的股票:
['CCC']
关空仓: BBB
新开多仓/调仓: AAA
新开空仓/调仓: CCC
----------------------
----------------------
2023-10-21: 
我目前持仓情况:
多头仓位: ['AAA']
空头仓位: ['CCC']
因子排名前1的股票:
['DDD']
因子排名后1的股票:
['BBB']
关多仓: AAA
关空仓: CCC
新开多仓/调仓: DDD
新开空仓/调仓: BBB
----------------------
所有未平仓持仓已经清算完毕！
回测结束，总资金: 20833.333333333332
